In [ ]:
#load the dataset
import pandas as pd
import numpy as np
df=pd.read_csv("/Users/sunilravilla/Documents/Work/7641/Project/Project_midterm/Hotel_Reviews.csv")
df.head()
required_columns=['Hotel_Address','Average_Score','Hotel_Name','Negative_Review','Positive_Review','Total_Number_of_Reviews','Reviewer_Score','lat','lng']
#filter the dataset to include only the required columns
df2=df[required_columns]
df2.head()
#replace No Negative with empty string in Negative_Review column
df2['Negative_Review']=df2['Negative_Review'].replace('No Negative', '')
df2['Positive_Review']=df2['Positive_Review'].replace('No Positive', '')
df2.head()
df.head()


In [ ]:
#Install pyabsa
!pip install pyabsa

In [ ]:
from pyabsa import ATEPCCheckpointManager

aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                                               auto_device=True  # False means load model on CPU
                                                               )


In [ ]:
#function to calculate the aspect based sentiment score
def aspect_sentiment(s):
    #s = 'Staff was very rude but food was delicious.The food was bad, but the service was good'
    atepc_results = []
    examples = [s]
    inference_source = examples
    atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                                                   pred_sentiment=True, )  # Predict the sentiment of extracted aspect terms

    #replace the sentiment with positive as 1 and negative as -1 and neutral as 0 in the sentiment dictionary

    atepc_result[0]['sentiment'] = [1 if i == 'Positive' else -1 if i=='Negative' else 0 for i in atepc_result[0]['sentiment']]
    
    temp_dict = {a.lower(): b*c for a, b,
                 c in zip(atepc_result[0]['aspect'], atepc_result[0]['sentiment'], atepc_result[0]['confidence'])}
    return temp_dict


#test the function
# s = 'Staff was very rude but food was delicious.The food was bad, but the service was good'
# aspect_sentiment(s)


In [ ]:
df2.head()
#Filter all rows with Hotel_name as Hotel Arena
#random sample of 1000 rows
df20=df2
list_of_hotels=df20['Hotel_Name'].unique()
from collections import defaultdict
master_dict = defaultdict(dict)

for i in range(650,len(list_of_hotels)):
    # Track the progress
    print("Hotel number in the list: ", i)
    dict_hotel = defaultdict(float)
    df_temp=df20[df20['Hotel_Name']==list_of_hotels[i]]
  
    
    for j in range(df_temp.shape[0]):
        #track the progress within the hotel
        print("Review number: ", j, "out of ", df_temp.shape[0])
        temp_dict_1 = aspect_sentiment(df_temp.iloc[j]['Positive_Review'])
        temp_dict_2 = aspect_sentiment(df_temp.iloc[j]['Negative_Review'])
        for key, value in temp_dict_1.items():
            dict_hotel[key] += value
        for key, value in temp_dict_2.items():
            dict_hotel[key] += value
    master_dict[list_of_hotels[i]] = dict(dict_hotel)
    #write the dictionary to a pickle file with the hotel name as the key and for hotel name as file name
    import pickle
    with open(list_of_hotels[i]+'.pkl', 'wb') as f:
        pickle.dump(dict_hotel, f)
    print(list_of_hotels[i])
    

dict(master_dict)




In [ ]:
#write the master dictionary to a pickle file
import pickle
with open('master_dict.pickle', 'wb') as handle:
    pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#read the pickle file
with open('master_dict.pickle', 'rb') as handle:
    master_dict = pickle.load(handle)

In [ ]:
#Check file type is .pkl and not .pickle if pickle file  then load the pickle file and form nested dictionary
import os
import pickle
master_dict={}
i=0
for file in os.listdir():
    if file.endswith(".pkl"):
        #track the progress
        print (i)
        i+=1
        with open(file, 'rb') as f:
            master_dict[file.split('.')[0]] = pickle.load(f)
dict(master_dict)

In [ ]:
#write the master dictionary to a pickle file
import pickle
with open('master_dict.pickle', 'wb') as handle:
    pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#extract all the keys from the nested dictionary into a list
keys=[]
for key, value in master_dict.items():
    for k, v in value.items():
        keys.append(k)
keys=list(set(keys))
print(len(keys))

#write the keys to a pickle file
import pickle
with open('Aspects.pickle', 'wb') as handle:
    pickle.dump(keys, handle, protocol=pickle.HIGHEST_PROTOCOL)

#read the pickle file into a list
with open('Aspects.pickle', 'rb') as handle:
    Aspects = pickle.load(handle)

print(keys[:200])

In [ ]:
import pandas as pd
d=master_dict
df21 = pd.DataFrame.from_dict({(i, j) : {'confidence':d[i][j] for i in d.keys() for j in d[i].keys()} for i in d.keys() for j in d[i].keys()}, orient = 'index')


In [ ]:
df21.head()
df21.to_csv('Hotel_name_aspect_and_sentiment.csv')

In [ ]:
#find the top 10 aspects for each hotel
df22=df21.groupby(level=0).apply(lambda x: x.sort_values('confidence', ascending=False).head(10))
df22.head(1000)


In [ ]:
#assign index names to the dataframe
df22.index.names = ['Hotel_Name', 'Aspect']


In [ ]:
#write the top 10 aspects to a csv file
df22.to_csv('Hotel_name_top_10_aspects.csv')

In [ ]:

from math import cos, asin, sqrt, pi


def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

    #test the function
distance(52.2296756, 21.0122287, 52.406374, 21.9251681)

In [ ]:
#generate two aspect values with random values
#intialise empty dataframe
df23 = pd.DataFrame(
    columns=['Hotel_Name', 'Aspect_1', 'Aspect_2', 'Aspect_3', 'Aspect_4', 'Aspect_5' ,'lat', 'lng', ])
df23['Aspect_1'] = np.random.randint(1, 40, df2.shape[0])
df23['Aspect_2'] = np.random.randint(1, 50, df2.shape[0])
df23['Aspect_3'] = np.random.randint(1, 35, df2.shape[0])
df23['Aspect_4'] = np.random.randint(1, 45, df2.shape[0])
df23['Aspect_5'] = np.random.randint(1, 70, df2.shape[0])
df23['Hotel_Name'] = df2['Hotel_Name']
df23['lat'] = df2['lat']
df23['lng'] = df2['lng']
df23.head()

#grouby hotel name and calculate the mean of the aspect values ,lat and lng , reset the index and assign to a new dataframe
df24=df23.groupby('Hotel_Name').mean().reset_index()
df24.head()

#write the dataframe to a csv file
df24.to_csv('Hotel_name_sample_data_for_GUI.csv')


In [ ]:
df24.head(100)
